In [1]:
# Installation des librairies nécessaires
!pip install datasets transformers[torch] accelerate -q
# Librairie pour la métrique d'évaluation ROUGE
!pip install rouge_score -q

print("Installation terminée.")

Installation terminée.


In [2]:
import re
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer

# --- PARAMÈTRE PRINCIPAL DU NOTEBOOK ---
# Changez cette variable pour tester un autre modèle (ex: "plguillou/t5-base-fr")
model_checkpoint = "moussaKam/barthez" 

print(f"Modèle choisi pour cette session : {model_checkpoint}")

Modèle choisi pour cette session : moussaKam/barthez


In [3]:
# Chargement du jeu de données
print("Chargement du jeu de données PleIAs/French-PD-Books...")
ds = load_dataset("PleIAs/French-PD-Books", split="train")
print("Jeu de données chargé.")
print(f"Nombre d'exemples avant nettoyage : {len(ds)}")

# --- Vos fonctions de nettoyage et de vérification ---
def clean_text(example):
    text = example["complete_text"]
    date = example.get("date", None)

    # Si la date contient un "-", on essaie d'extraire l'année connue ou moyenne
    if date and "-" in str(date):
        parts = str(date).split("-")
        if (parts[1].isdigit() and len(parts[1]) == 4) and parts[0] == "????":
            date = str(parts[1])
        else:
            date = str(parts[0])

    # Appliquer les nettoyages seulement si le texte est une chaîne de caractères
    if isinstance(text, str):
        # 1. Retirer les numéros de page
        text = re.sub(r"[—\-–]\s*\d+\s*[—\-–]", " ", text)
        # 2. Corriger les caractères échappés
        text = text.replace("\\'", "'").replace("\\\"", "\"").replace("\\n", " ").replace("\\r", " ").replace("\\t", " ")
        # 3. & 4. Corriger les mots coupés et espaces multiples (simplifié)
        text = re.sub(r'([a-zàâäæçéèêëïîôùûüœ])\s{2,}([a-zàâäæçéèêëïîôùûüœ])', r'\1 \2', text)
        # 5. Normaliser les espaces
        text = re.sub(r"\s+", " ", text)
        # 6. Nettoyer les caractères spéciaux
        text = re.sub(r"[^\w\s\.,;:\?!'\-\"«»À-ÖØ-öø-ÿœŒ]", " ", text)
        # 7. Re-normaliser
        text = re.sub(r"\s+", " ", text)
        # 8. Corriger la ponctuation
        text = re.sub(r"\s+([,.\?!;:])", r"\1", text)
        text = re.sub(r"([,.\?!;:])\s*([,.\?!;:])", r"\1\2", text)
        text = text.strip()
    else:
        text = "" # Si le texte n'est pas une chaîne, le remplacer par une chaîne vide

    return {"text": text, "date": str(date)}

# Application de la fonction de nettoyage
print("\nApplication du nettoyage sur le dataset...")
reduced_ds = ds.shuffle(seed=42).select(range(1000))
ds_cleaned = reduced_ds.map(clean_text, num_proc=4) # Utilise plusieurs processeurs pour accélérer

# Filtrage des données invalides (dates incorrectes ou textes vides)
print("Filtrage des données invalides...")
ds_filtered = ds_cleaned.filter(lambda example: 
    example['date'].isdigit() and 
    len(example['date']) == 4 and 
    len(example['text']) > 50 # Garder des textes d'une longueur minimale
)

print(f"Nombre d'exemples après nettoyage et filtrage : {len(ds_filtered)}")

# Affichage d'un exemple pour vérification
print("\n--- Exemple après nettoyage ---")
example = ds_filtered[500]
print(f"Date: {example['date']}")
print(f"Texte: {example['text'][:400]}")

Chargement du jeu de données PleIAs/French-PD-Books...


Resolving data files:   0%|          | 0/145 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/145 [00:00<?, ?it/s]

Jeu de données chargé.
Nombre d'exemples avant nettoyage : 289577

Application du nettoyage sur le dataset...
Filtrage des données invalides...
Nombre d'exemples après nettoyage et filtrage : 1000

--- Exemple après nettoyage ---
Date: 1923
Texte: MINISTÈRE DES AFFAIRES ÉTRANGÈRES DOCUMENTS DIPLOMATIQUES REPONSE DU GOUVERNEMENT FRANÇAIS À LA LETTRE DU GOUVERNEMENT BRITANNIQUE DU il AOÛT 1923 SUR LES RÉPARATIONS 20 AOUT 1923 PARIS IMPRIMERIE NATIONALE MDCGCCXMII MINISTÈRE DES AFFAIRES ÉTRANGÈRES DOCUMENTS DIPLOMATIQUES REPONSE DU GOUVERNEMENT FRANÇAIS À LA LETTRE DU GOUVERNEMENT RRITANNIQUE DU 11 AOÛT 1923 SUR LES RÉPARATIONS 20 AOUT 1923 LI


In [4]:
# Fonction pour créer les entrées et sorties du modèle
def create_seq2seq_format(example):
    text = example['text']
    year = int(example['date'])
    
    # Création de la décennie (ex: 1854 -> 1850)
    decade = (year // 10) * 10
    
    # Formatage de l'entrée pour le modèle
    # Le préfixe est crucial pour que le modèle comprenne la tâche
    prefix = f"réécris ce texte dans le style des années {decade}: "
    
    # Source : ce que le modèle voit en entrée
    example['source'] = prefix + text
    # Cible : ce que le modèle doit apprendre à prédire
    example['target'] = text
    
    return example

print("\nFormatage des données pour la tâche sequence-to-sequence...")
ds_seq2seq = ds_filtered.map(create_seq2seq_format, num_proc=4)

print("Formatage terminé.")
print("\n--- Exemple au format Seq2Seq ---")
example = ds_seq2seq[500]
print(f"SOURCE (entrée modèle):\n{example['source'][:500]}\n")
print(f"TARGET (sortie attendue):\n{example['target'][:400]}")


Formatage des données pour la tâche sequence-to-sequence...
Formatage terminé.

--- Exemple au format Seq2Seq ---
SOURCE (entrée modèle):
réécris ce texte dans le style des années 1920: MINISTÈRE DES AFFAIRES ÉTRANGÈRES DOCUMENTS DIPLOMATIQUES REPONSE DU GOUVERNEMENT FRANÇAIS À LA LETTRE DU GOUVERNEMENT BRITANNIQUE DU il AOÛT 1923 SUR LES RÉPARATIONS 20 AOUT 1923 PARIS IMPRIMERIE NATIONALE MDCGCCXMII MINISTÈRE DES AFFAIRES ÉTRANGÈRES DOCUMENTS DIPLOMATIQUES REPONSE DU GOUVERNEMENT FRANÇAIS À LA LETTRE DU GOUVERNEMENT RRITANNIQUE DU 11 AOÛT 1923 SUR LES RÉPARATIONS 20 AOUT 1923 LIVRE «.UNE. Réponse du Gouv. français. MINISTÈRE DES 

TARGET (sortie attendue):
MINISTÈRE DES AFFAIRES ÉTRANGÈRES DOCUMENTS DIPLOMATIQUES REPONSE DU GOUVERNEMENT FRANÇAIS À LA LETTRE DU GOUVERNEMENT BRITANNIQUE DU il AOÛT 1923 SUR LES RÉPARATIONS 20 AOUT 1923 PARIS IMPRIMERIE NATIONALE MDCGCCXMII MINISTÈRE DES AFFAIRES ÉTRANGÈRES DOCUMENTS DIPLOMATIQUES REPONSE DU GOUVERNEMENT FRANÇAIS À LA LETTRE DU GOUVERNEM

In [5]:
import os
from tqdm.auto import tqdm
from datasets import Dataset

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Chargement du tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Longueurs maximales pour les séquences source et cible
# À ajuster en fonction de la mémoire disponible et des capacités du modèle
max_input_length = 512
max_target_length = 512

def tokenize_function(examples):
    # Tokenisation des textes sources
    model_inputs = tokenizer(
        examples["source"], 
        max_length=max_input_length, 
        truncation=True, 
        padding="max_length"
    )

    # Tokenisation des textes cibles pour les labels
    labels = tokenizer(
        text_target=examples["target"], 
        max_length=max_target_length, 
        truncation=True, 
        padding="max_length"
    )

    # Assigner les labels tokenisés à la clé 'labels' que le modèle attend
    model_inputs["labels"] = labels["input_ids"]
    
    # Remplacer les tokens de padding dans les labels par -100 pour qu'ils soient ignorés dans le calcul de la loss
    for i, label_ids in enumerate(model_inputs["labels"]):
        model_inputs["labels"][i] = [l if l != tokenizer.pad_token_id else -100 for l in label_ids]

    return model_inputs

print(f"\nLancement de la tokenisation avec le tokenizer de '{model_checkpoint}'...")

# On va tester sur les 1000 premiers exemples
num_to_test = 1000
problematic_index = -1

print(f"\n--- Traitement manuel des {num_to_test} premiers exemples ---")

# La barre de progression (tqdm) nous montrera exactement où ça bloque
# Liste pour stocker les résultats tokenisés
tokenized_results = []

# On utilise l'intégralité du dataset
num_to_process = len(ds_seq2seq) 

print(f"--- Traitement et tokenisation des {num_to_process} exemples ---")

# La boucle parcourt tout le dataset
for i in tqdm(range(num_to_process)):
    # On récupère un exemple
    example = ds_seq2seq[i]
    
    # On le met dans un format de "batch" avec un seul élément
    batch = {
        "source": [example["source"]],
        "target": [example["target"]]
    }
    
    # On applique la fonction de tokenisation
    tokenized_output = tokenize_function(batch)
    
    # On "déballe" le résultat du batch pour n'avoir qu'un seul dictionnaire
    # et on l'ajoute à notre liste de résultats
    tokenized_results.append({
        'input_ids': tokenized_output['input_ids'][0],
        'attention_mask': tokenized_output['attention_mask'][0],
        'labels': tokenized_output['labels'][0]
    })

print("\n✅ Tokenisation manuelle terminée.")

# Conversion de la liste de dictionnaires en un objet Dataset de Hugging Face
ds_tokenized = Dataset.from_list(tokenized_results)




Lancement de la tokenisation avec le tokenizer de 'moussaKam/barthez'...

--- Traitement manuel des 1000 premiers exemples ---
--- Traitement et tokenisation des 1000 exemples ---


  0%|          | 0/1000 [00:00<?, ?it/s]


✅ Tokenisation manuelle terminée.


In [6]:
# Fractionnement du jeu de données : 90% pour l'entraînement, 10% pour la validation
ds_final = ds_tokenized.train_test_split(test_size=0.1)

# Renommage pour plus de clarté
ds_final["train"] = ds_final.pop("train")
ds_final["validation"] = ds_final.pop("test")

print("Taille du jeu d'entraînement :", len(ds_final["train"]))
print("Taille du jeu de validation :", len(ds_final["validation"]))
print(ds_final)

Taille du jeu d'entraînement : 900
Taille du jeu de validation : 100
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 900
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})


In [7]:
%pip install evaluate -q

import numpy as np
import evaluate # Librairie de Hugging Face pour les métriques

# Chargement de la métrique ROUGE
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Décodage des prédictions générées par le modèle
    # On remplace les tokens -100 (ignorés dans la loss) par le token de padding
    preds[preds == -100] = tokenizer.pad_token_id
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Décodage des vrais labels de référence
    labels[labels == -100] = tokenizer.pad_token_id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Calcul du score ROUGE
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    # Ajout d'une métrique sur la longueur moyenne des prédictions
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

print("Fonction de calcul des métriques définie.")

Note: you may need to restart the kernel to use updated packages.
Fonction de calcul des métriques définie.


In [ ]:
# Install/upgrade all libraries in one command for better dependency resolution
%pip install -U bitsandbytes accelerate peft transformers torch -q

import torch
import os
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Isolate your script on the GPU 1
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# --- 1. Configuration de la Quantization (QLoRA) ---
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# --- 2. Chargement du Modèle quantizé ---
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint,
    quantization_config=quantization_config,
    device_map="auto"
)

print("Modèle chargé en 4-bit (QLoRA) sur le GPU 1.")

# --- 3. Préparation du modèle et Configuration de LoRA ---
model = prepare_model_for_kbit_training(model)

# ## CORRECTIF ##: Ciblez toutes les couches linéaires pour assurer un chemin de gradient complet.
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# --- Le reste du code reste identique ---

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-qlora-finetuned-style-transfer"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    predict_with_generate=True,
    fp16=False,
    bf16=True,
    push_to_hub=False,
)

# On retire l'argument 'tokenizer' qui est obsolète
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=ds_final["train"],
    eval_dataset=ds_final["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Entraîneur (Trainer) prêt pour l'entraînement avec QLoRA sur le GPU 1.")

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Lancement de l'entraînement !
print("🚀 Début de l'entraînement...")
trainer.train()
print("✅ Entraînement terminé !")

# Sauvegarde du meilleur modèle dans le dossier de sortie
trainer.save_model()
print(f"Meilleur modèle sauvegardé dans le dossier : {output_dir}")

🚀 Début de l'entraînement...


/opt/conda/lib/python3.13/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  first_ctx: bool = False,
/opt/conda/lib/python3.13/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
# Évaluation finale du meilleur modèle sur le jeu de validation
print("\nÉvaluation finale du modèle...")
eval_results = trainer.evaluate()

print("\n--- Résultats de l'évaluation ---")
for key, value in eval_results.items():
    print(f"{key}: {value}")